<h2><center>Train network</center></h2>

(c) DI Dominik Hirner BSc. 
Institute for graphics and vision (ICG)
University of Technology Graz, Austria
E-mail: dominik.hirner@tugraz.at

This notebook is equivalent to train.py in the root of this repository

In [ ]:
import sys
import numpy as np
import cv2
import re
import numpy.matlib
import torch
import torch.nn as nn
from torch.autograd import Variable
from PIL import Image
from typing import Tuple
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch import optim

from guided_filter_pytorch.guided_filter import GuidedFilter
import argparse
import glob
import random

In [ ]:
#MB, KITTI2012, KITTI2015 or ETH
dataset = 'KITTI2015'
#used as prefix for saved weights
model_name = 'mb_64f'

input_folder = '/media/HDD/TrainingsData/kitti2015/training/'

#patches per batch
batch_size = 40
#batches per epoch
nr_batches = 10
nr_epochs = 20000000
num_conv_feature_maps = 64
#save weights every x iteration
save_weights = 1
#crop size for patch
patch_size = 21
ps_h = int(patch_size/2)
#range for offset of o_neg
r_low = 1
r_high = 25

In [ ]:
def loadMB():
    
    left_filelist = glob.glob(input_folder + '*/im0.png')
    right_filelist = glob.glob(input_folder + '*/im1.png')
    disp_filelist = glob.glob(input_folder + '*/disp0GT.pfm')
    
    left_filelist = sorted(left_filelist)
    right_filelist = sorted(right_filelist)
    disp_filelist = sorted(disp_filelist)
    
    left_list = []
    right_list = []
    disp_list = []
    
    for i in range(0,len(left_filelist)):
        
        cur_left = cv2.imread(left_filelist[i])
        cur_right = cv2.imread(right_filelist[i])
        cur_disp,_ = readPFM(disp_filelist[i])
        
        left_list.append(cur_left)
        right_list.append(cur_right)
        disp_list.append(cur_disp)
        
    return left_list, right_list, disp_list

In [ ]:
def loadETH3D():
    
    left_filelist = glob.glob(input_folder + '*/im0.png')
    right_filelist = glob.glob(input_folder + '*/im1.png')
    disp_filelist = glob.glob(input_folder + '*/disp0GT.pfm')
    
    left_filelist = sorted(left_filelist)
    right_filelist = sorted(right_filelist)
    disp_filelist = sorted(disp_filelist)
    
    left_list = []
    right_list = []
    disp_list = []
    
    for i in range(0,len(left_filelist)):
        
        cur_left = cv2.imread(left_filelist[i])
        cur_right = cv2.imread(right_filelist[i])
        cur_disp,_ = readPFM(disp_filelist[i])
        
        left_list.append(cur_left)
        right_list.append(cur_right)
        disp_list.append(cur_disp)
        
    return left_list, right_list, disp_list

In [ ]:
def loadKitti2012():

    left_filelist = glob.glob(input_folder + 'colored_0/*.png')
    right_filelist = glob.glob(input_folder + 'colored_1/*.png')
    disp_filelist = glob.glob(input_folder + 'disp_noc/*.png')
    
    left_filelist = sorted(left_filelist)
    right_filelist = sorted(right_filelist)
    disp_filelist = sorted(disp_filelist)

    left_elem_list = []
    for left_im in left_filelist:

        left_im_el = left_im.split('/')[-1]
        left_elem_list.append(left_im_el)

    left_elem_list = sorted(left_elem_list)

    right_elem_list = []
    for right_im in right_filelist:

        right_im_el = right_im.split('/')[-1]
        right_elem_list.append(right_im_el)

    right_elem_list = sorted(right_elem_list)

    gt_elem_list = []
    for gt_im in disp_filelist:

        gt_im_el = gt_im.split('/')[-1]
        gt_elem_list.append(gt_im_el)

    gt_elem_list = sorted(gt_elem_list)
    inters_list = set(left_elem_list) & set(right_elem_list) & set(gt_elem_list)
   
    inters_list = list(inters_list)
    left_list = []
    right_list = []
    disp_list = []
    
    for i in range(0,len(inters_list)):
        
        left_im = input_folder + 'colored_0/' + inters_list[i]
        right_im = input_folder + 'colored_1/' + inters_list[i]
        disp_im =  input_folder + 'disp_noc/' + inters_list[i] 
       
        cur_left = cv2.imread(left_im)
        cur_right = cv2.imread(right_im)
        cur_disp = cv2.imread(disp_im)
        
        cur_disp = np.mean(cur_disp,axis=2) 
        
        left_list.append(cur_left)
        right_list.append(cur_right)
        disp_list.append(cur_disp)
        
    return left_list, right_list, disp_list

In [ ]:
def loadKitti2015():

    left_filelist = glob.glob(input_folder + 'image_2/*.png')
    right_filelist = glob.glob(input_folder + 'image_3/*.png')
    disp_filelist = glob.glob(input_folder + 'disp_noc_0/*.png')
    
    left_filelist = sorted(left_filelist)
    right_filelist = sorted(right_filelist)
    disp_filelist = sorted(disp_filelist)

    left_elem_list = []
    for left_im in left_filelist:

        left_im_el = left_im.split('/')[-1]
        left_elem_list.append(left_im_el)

    left_elem_list = sorted(left_elem_list)


    right_elem_list = []
    for right_im in right_filelist:

        right_im_el = right_im.split('/')[-1]
        right_elem_list.append(right_im_el)

    right_elem_list = sorted(right_elem_list)



    gt_elem_list = []
    for gt_im in disp_filelist:

        gt_im_el = gt_im.split('/')[-1]
        gt_elem_list.append(gt_im_el)

    gt_elem_list = sorted(gt_elem_list)


    inters_list = set(left_elem_list) & set(right_elem_list) & set(gt_elem_list)
   
    inters_list = list(inters_list)
    left_list = []
    right_list = []
    disp_list = []
    
    for i in range(0,len(inters_list)):
        
        left_im = input_folder + 'image_2/' + inters_list[i]
        right_im = input_folder + 'image_3/' + inters_list[i]
        disp_im =  input_folder + 'disp_noc_0/' + inters_list[i] 
       
        cur_left = cv2.imread(left_im)
        cur_right = cv2.imread(right_im)
        cur_disp = cv2.imread(disp_im)
        
        cur_disp = np.mean(cur_disp,axis=2) 
        
        left_list.append(cur_left)
        right_list.append(cur_right)
        disp_list.append(cur_disp)
        
    return left_list, right_list, disp_list

In [ ]:
class SiameseBranch(nn.Module):
    def __init__(self,img_ch=3):
        super(SiameseBranch,self).__init__()
        
        self.Tanh = nn.Tanh()        
        self.Conv1 = nn.Conv2d(img_ch, num_conv_feature_maps, kernel_size = 3,stride=1,padding = 1,dilation = 1, bias=True)
        self.Conv2 = nn.Conv2d(num_conv_feature_maps, num_conv_feature_maps, kernel_size=3,stride=1,padding = 1,dilation = 1, bias=True)
        self.Conv3 = nn.Conv2d(2*num_conv_feature_maps, num_conv_feature_maps, kernel_size=3,stride=1,padding = 1,dilation = 1, bias=True)
        self.Conv4 = nn.Conv2d(3*num_conv_feature_maps, num_conv_feature_maps, kernel_size=3,stride=1,padding = 1,dilation = 1,bias=True)
        self.Conv5 = nn.Conv2d(4*num_conv_feature_maps, num_conv_feature_maps, kernel_size=3,stride=1,padding = 1,dilation = 1, bias=True)        
        
    def forward(self,x_in):
        
        x1 = self.Conv1(x_in) 
        x1 = self.Tanh(x1)
                
        x2 = self.Conv2(x1) 
        x2 = self.Tanh(x2)
        
        d2 = torch.cat((x1,x2),dim=1)
        
        x3 = self.Conv3(d2) 
        x3 = self.Tanh(x3)
        
        d3 = torch.cat((x1,x2,x3),dim=1)
        
        x4 = self.Conv4(d3)
        x4 = self.Tanh(x4)
        
        d4 = torch.cat((x1,x2,x3,x4),dim=1)
        
        x5 = self.Conv5(d4)
        
        return x5

In [ ]:
branch = SiameseBranch()
branch = branch.cuda()

In [ ]:
pytorch_total_params = sum(p.numel() for p in branch.parameters() if p.requires_grad)
print("Nr feat: " ,pytorch_total_params)

In [ ]:
##python3 version!!!!
def readPFM(file):
    file = open(file, 'rb')

    color = None
    width = None
    height = None
    scale = None
    endian = None

    header = file.readline().decode('utf-8').rstrip()
    if header == 'PF':
        color = True
    elif header == 'Pf':
        color = False
    else:
        raise Exception('Not a PFM file.')

    dim_match = re.match(r'^(\d+)\s(\d+)\s$', file.readline().decode('utf-8'))
    if dim_match:
        width, height = map(int, dim_match.groups())
    else:
        raise Exception('Malformed PFM header.')

    scale = float(file.readline().decode('utf-8').rstrip())
    if scale < 0:  # little-endian
        endian = '<'
        scale = -scale
    else:
        endian = '>'  # big-endian

    data = np.fromfile(file, endian + 'f')
    shape = (height, width, 3) if color else (height, width)

    data = np.reshape(data, shape)
    data = np.flipud(data)
    return data, scale

In [ ]:
def writePFM(file, image, scale=1):
    file = open(file, 'wb')

    color = None

    if image.dtype.name != 'float32':
        raise Exception('Image dtype must be float32.')

    image = np.flipud(image)

    if len(image.shape) == 3 and image.shape[2] == 3:  # color image
        color = True
    elif len(image.shape) == 2 or len(image.shape) == 3 and image.shape[2] == 1:  # greyscale
        color = False
    else:
        raise Exception('Image must have H x W x 3, H x W x 1 or H x W dimensions.')

    file.write('PF\n'.encode() if color else 'Pf\n'.encode())
    file.write('%d %d\n'.encode() % (image.shape[1], image.shape[0]))

    endian = image.dtype.byteorder

    if endian == '<' or endian == '=' and sys.byteorder == 'little':
        scale = -scale

    file.write('%f\n'.encode() % scale)

    image.tofile(file)

In [ ]:
Tensor = torch.cuda.FloatTensor
cos = torch.nn.CosineSimilarity()

In [ ]:
def createCostVol(left_im,right_im,max_disp):
    
    a_h, a_w,c = left_im.shape
    left_im = np.transpose(left_im, (2,0,1)).astype(np.uint8)
    right_im = np.transpose(right_im, (2,0,1)).astype(np.uint8)
    
    left_im = np.reshape(left_im, [1,c,a_h,a_w])
    right_im = np.reshape(right_im, [1,c,a_h,a_w])
    
    with torch.no_grad():

        left_imT = Variable(Tensor(left_im.astype(np.uint8)))
        right_imT = Variable(Tensor(right_im.astype(np.uint8)))

        left_feat = branch(left_imT)
        right_feat = branch(right_imT)
        
        _,f,h,w = left_feat.shape
        
        cost_vol = np.zeros((max_disp+1,a_h,a_w))
        cost_volT = Variable(Tensor(cost_vol))   

        #0 => max_disp => one less disp!
        for disp in range(0,max_disp+1):
            if(disp == 0):
                sim_score = cos(left_feat, right_feat)
                cost_volT[disp,:,:] = torch.squeeze(sim_score) 
            else:
                right_shifted = torch.cuda.FloatTensor(1,f,h,w).fill_(0)                      
                right_shift = torch.cuda.FloatTensor(1,f,h,disp).fill_(0)  
                right_appended = torch.cat([right_shift,right_feat],3)

                _,f,h_ap,w_ap = right_appended.shape
                right_shifted[:,:,:,:] = right_appended[:,:,:,:(w_ap-disp)]
                sim_score = cos(left_feat, right_shifted)
                cost_volT[disp,:,:] = torch.squeeze(sim_score)              
    return cost_volT

In [ ]:
def getBatch():
    
    batch_xl = np.zeros((batch_size,3,patch_size,patch_size))
    batch_xr_pos = np.zeros((batch_size,3,patch_size,patch_size))
    batch_xr_neg = np.zeros((batch_size,3,patch_size,patch_size))
    
    for el in range(batch_size):
        
        if(el % 25 == 0):
            
            ridx = np.random.randint(0,len(left_list),1)
            left_im = left_list[ridx[0]]
            right_im = right_list[ridx[0]]
            gt_im = gt_list[ridx[0]]

        
        #get random position
        h,w,c = left_im.shape
        r_h = 0
        r_w = 0
        d = 0
#        print('Draw for random position')
        #also check height! should not draw corner pixels!!
        while True:
            r_h = random.sample(range(ps_h,h-(ps_h+1)), 1)
            r_w = random.sample(range(ps_h,w-(ps_h+1)),1)            
            if(not np.isinf(gt_im[r_h,r_w])):
                d = int(np.round(gt_im[r_h,r_w]))
                if((r_w[0]-ps_h-d-1) >= 0):
                     if((r_w[0]+(ps_h+1)-d+1) <= w):
                        break
        
        d = int(np.round(gt_im[r_h,r_w]))
                
        cur_left = left_im[r_h[0]-ps_h:r_h[0]+(ps_h+1), r_w[0]-ps_h:r_w[0]+(ps_h+1),:]
        #choose offset
        
        o_pos = 0                
        cur_right_pos = right_im[r_h[0]-ps_h:r_h[0]+(ps_h+1), (r_w[0]-ps_h-d+o_pos):(r_w[0]+(ps_h+1)-d+o_pos),:]

        
        #should not be too close to real match!
        o_neg = 0
        while True:
            #range 6-8??? range(2,6)
            o_neg = random.sample(range(r_low,r_high), 1)
            if np.random.randint(-1, 1) == -1:
                o_neg = -o_neg[0]
            else:
                o_neg = o_neg[0]
            #try without d-+1   and(o_neg != (d-1)) and(o_neg != (d+1))
            if((o_neg != d) and ((r_w[0]-ps_h-d+o_neg) > 0)  and ((r_w[0]+(ps_h+1)-d+o_neg) < w)):
                break
        
        
        cur_right_neg = right_im[r_h[0]-ps_h:r_h[0]+(ps_h+1), (r_w[0]-ps_h-d+o_neg):(r_w[0]+(ps_h+1)-d+o_neg),:]

        
        batch_xl[el,:,:,:] =  np.transpose(cur_left, (2,0,1)).astype(np.uint8)
        batch_xr_pos[el,:,:,:] = np.transpose(cur_right_pos, (2,0,1)).astype(np.uint8)
        batch_xr_neg[el,:,:,:] = np.transpose(cur_right_neg, (2,0,1)).astype(np.uint8)
            
    return batch_xl, batch_xr_pos, batch_xr_neg


In [ ]:
pytorch_total_params = sum(p.numel() for p in branch.parameters() if p.requires_grad)
print("Nr feat: " ,pytorch_total_params)

In [ ]:
def my_hinge_loss(s_p, s_n):
    margin = 0.2
    relu = torch.nn.ReLU()
    relu = relu.cuda()
    loss = relu(-((s_p - s_n) - margin))

    return loss

In [ ]:
if(dataset == 'KITTI2012'):
    left_list, right_list, gt_list = loadKitti2012()
if(dataset == 'KITTI2015'):
    left_list, right_list, gt_list = loadKitti2015()
if(dataset == 'MB'):
    left_list, right_list, gt_list = loadMB()    
if(dataset == 'ETH'):
    left_list, right_list, gt_list = loadETH3D()

In [ ]:
nr_samples = len(left_list)

In [ ]:
#If you want to use pre-trained weights as a starting point
#load them here!
#branch.load_state_dict(torch.load('/media/HDD/FC-DCNN-githubICPR/eth_13700'))

In [ ]:
optimizer_G = optim.Adam(branch.parameters(), lr=0.00006)

left_patches = []
right_pos_patches = []
right_neg_patches = []

loss_list = []

for i in range(nr_epochs):
            
    epoch_loss = 0.0        
    for cur_batch in range(batch_size):
         #reset gradients
        optimizer_G.zero_grad()

        batch_xl, batch_xr_pos, batch_xr_neg = getBatch()
        bs, c, h, w = batch_xl.shape
        batch_loss = 0.0

        xl = Variable(Tensor(batch_xl.astype(np.uint8)))
        xr_pos = Variable(Tensor(batch_xr_pos.astype(np.uint8)))
        xr_neg = Variable(Tensor(batch_xr_neg.astype(np.uint8)))        

        left_out = branch(xl)
        right_pos_out = branch(xr_pos)
        right_neg_out = branch(xr_neg)
        
        sp = cos(left_out, right_pos_out)
        sn = cos(left_out, right_neg_out)            
        
        batch_loss = my_hinge_loss(sp, sn)
        batch_loss = batch_loss.mean()      

        batch_loss.backward()
        optimizer_G.step()
        epoch_loss = epoch_loss + batch_loss
    
    epoch_loss = batch_loss/nr_batches        
    if(i % save_weights == 0):
        torch.save(branch.state_dict(), '../save_weights/' + model_name + '_%04i' %(i)) 
        print("EPOCH: {} loss: {}".format(i,epoch_loss))